In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import catboost
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/train.csv',na_values=['?','-999','Error','xxxxxxxx'])
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,NaN,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,NaN,CID21329,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,NaN,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [3]:
df.isnull().sum()

customer_id                         0
Name                                0
age                                 0
gender                              0
security_no                         0
region_category                  5428
membership_category                 0
joining_date                        0
joined_through_referral          5438
referral_id                     17846
preferred_offer_types             288
medium_of_operation              5393
internet_option                     0
last_visit_time                     0
days_since_last_login            1999
avg_time_spent                      0
avg_transaction_value               0
avg_frequency_login_days         3522
points_in_wallet                 3443
used_special_discount               0
offer_application_preference        0
past_complaint                      0
complaint_status                    0
feedback                            0
churn_risk_score                    0
dtype: int64

In [4]:
import pandas_profiling

In [5]:
# df.profile_report()

In [6]:
df.isnull().sum()

customer_id                         0
Name                                0
age                                 0
gender                              0
security_no                         0
region_category                  5428
membership_category                 0
joining_date                        0
joined_through_referral          5438
referral_id                     17846
preferred_offer_types             288
medium_of_operation              5393
internet_option                     0
last_visit_time                     0
days_since_last_login            1999
avg_time_spent                      0
avg_transaction_value               0
avg_frequency_login_days         3522
points_in_wallet                 3443
used_special_discount               0
offer_application_preference        0
past_complaint                      0
complaint_status                    0
feedback                            0
churn_risk_score                    0
dtype: int64

In [7]:
df.nunique()

customer_id                     36992
Name                            36992
age                                55
gender                              3
security_no                     36992
region_category                     3
membership_category                 6
joining_date                     1096
joined_through_referral             2
referral_id                     11358
preferred_offer_types               3
medium_of_operation                 3
internet_option                     3
last_visit_time                 30101
days_since_last_login              26
avg_time_spent                  25961
avg_transaction_value           36894
avg_frequency_login_days         1653
points_in_wallet                23699
used_special_discount               2
offer_application_preference        2
past_complaint                      2
complaint_status                    5
feedback                            9
churn_risk_score                    6
dtype: int64

In [8]:
df=df.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [9]:
df['churn_risk_score'].value_counts()

 3    10424
 4    10185
 5     9827
 2     2741
 1     2652
-1     1163
Name: churn_risk_score, dtype: int64

In [10]:
def fillNan(df, col, value):
    df[col].fillna(value, inplace=True)
    
fillNan(df, 'region_category', df['region_category'].mode()[0])
fillNan(df, 'points_in_wallet', df['points_in_wallet'].mean())
fillNan(df, 'preferred_offer_types', df['preferred_offer_types'].mode()[0])
# fillNan(df, 'joined_through_referral', 'No')
# fillNan(df, 'medium_of_operation', 'Desktop')

In [11]:
df.isnull().sum()

age                                0
gender                             0
region_category                    0
membership_category                0
joining_date                       0
joined_through_referral         5438
preferred_offer_types              0
medium_of_operation             5393
internet_option                    0
days_since_last_login           1999
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days        3522
points_in_wallet                   0
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [12]:
for i in df.columns:
  if df[i].dtype=='float64':
    df[i]=df[i].fillna(df[i].mean())
  else:
    df[i]=df[i].fillna(method='ffill')

In [13]:
df.isnull().sum().sum()

1

In [14]:
# df['joining_date']=df['joining_date'].apply(lambda x:int(x.split('-')[0]))
# df.rename(columns={'joining_date':'joining_year'},inplace=True)

df['joining_date'] = pd.to_datetime(df['joining_date'])
df['days_since_joined'] = df['joining_date'].apply(lambda x:(pd.Timestamp('today') - x).days)

df.drop(['joining_date'], inplace=True, axis=1)

In [15]:
df.head()

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,days_since_joined
0,18,F,Village,Platinum Membership,No,Gift Vouchers/Coupons,NaN,Wi-Fi,17.0,300.63,53005.25,17.0,781.750000,Yes,Yes,No,Not Applicable,Products always in Stock,2,1337
1,32,F,City,Premium Membership,No,Gift Vouchers/Coupons,Desktop,Mobile_Data,16.0,306.34,12838.38,10.0,686.882199,Yes,No,Yes,Solved,Quality Customer Care,1,1326
2,44,F,Town,No Membership,Yes,Gift Vouchers/Coupons,Desktop,Wi-Fi,14.0,516.16,21027.00,22.0,500.690000,No,Yes,Yes,Solved in Follow-up,Poor Website,5,1616
3,37,M,City,No Membership,Yes,Gift Vouchers/Coupons,Desktop,Mobile_Data,11.0,53.27,25239.56,6.0,567.660000,No,Yes,Yes,Unsolved,Poor Website,5,1629
4,31,F,City,No Membership,No,Credit/Debit Card Offers,Smartphone,Mobile_Data,20.0,113.13,24483.66,16.0,663.060000,No,Yes,Yes,Solved,Poor Website,5,1311


In [16]:
df1=pd.get_dummies(df,drop_first=True)

In [17]:
x=df1.drop('churn_risk_score',axis=1)
y=df1['churn_risk_score']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,stratify=y,random_state=42)

In [18]:
rd=RandomForestClassifier()

In [19]:
rd.fit(x_train,y_train)

RandomForestClassifier()

In [20]:
rdpred=rd.predict(x_test)

In [21]:
print(classification_report(y_test,rdpred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       116
           1       0.71      0.83      0.77       265
           2       0.78      0.71      0.74       274
           3       0.89      0.90      0.89      1043
           4       0.71      0.56      0.63      1019
           5       0.69      0.90      0.78       983

    accuracy                           0.76      3700
   macro avg       0.63      0.65      0.63      3700
weighted avg       0.74      0.76      0.74      3700



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
print(f1_score(y_test,rdpred,average='macro'))

0.6345864782783642


In [23]:
import xgboost

In [24]:
xgc=xgboost.XGBClassifier()

In [25]:
xgc.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:15:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [26]:
xgcpred=xgc.predict(x_test)

In [27]:
print(f1_score(y_test,xgcpred,average='macro'))

0.6244562711665188


In [28]:
# params={'learning_rate':[0.05,0.1,0.15,0.2],
#       'max_depth':[3,4,5,6,8,10],
#       'min_child_weight':[1,3,5,7],
#       'gamma':[0.0,0.1,0.2,0.3,0.4,0.5],
#       'colsample_bytree':[0.3,0.4,0.5,0.7]}
# rsearch=RandomizedSearchCV(xgc,param_distributions=params,n_iter=100,cv=3,random_state=42,n_jobs=-1)
# rsearch.fit(x,y)
# rsearch.best_parameter_ 

In [29]:
# best_estimaters=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4,
#              learning_rate=0.1, max_delta_step=0, max_depth=4,
#             min_child_weight=3, missing=None, n_estimators=119, n_jobs=1,
#             nthread=None, objective='multi:softprob', random_state=0,
#             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#             silent=None, subsample=1, verbosity=1)

In [30]:
xgc=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=None, n_estimators=119, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [31]:
xgc.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:15:42] WARNING: ../src/learner.cc:541: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:15:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=None, monotone_constraints='()',
              n_estimators=119, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=0, silent=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=1)

In [32]:
xgcpred=xgc.predict(x_test)

In [33]:
print(f1_score(y_test,xgcpred,average='macro'))

0.6331652806499237


In [34]:
xgc.fit(x,y)

[12:16:06] WARNING: ../src/learner.cc:541: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:16:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=None, monotone_constraints='()',
              n_estimators=119, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=0, silent=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=1)

In [35]:
test=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/test.csv',na_values=['?','-999','Error','xxxxxxxx'])
test.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440031003700300030003400,Alethia Meints,50,F,OQJ1XAY,Village,Premium Membership,2015-11-02,No,NaN,...,12.0,386.26,40721.44,7.0,733.830000,Yes,No,No,Not Applicable,Poor Product Quality
1,fffe43004900440031003900370037003300,Ming Lopez,41,M,OUQRPKO,Village,Gold Membership,2016-03-01,No,NaN,...,11.0,37.80,9644.40,9.0,726.000000,Yes,No,No,Not Applicable,Poor Website
2,fffe43004900440034003800360037003000,Carina Flannigan,31,F,02J2RE7,Town,Silver Membership,2017-03-03,No,NaN,...,18.0,215.36,3693.25,21.0,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified
3,fffe43004900440036003200370033003400,Kyung Wanner,64,M,5YEQIF1,Town,Silver Membership,2017-08-18,Yes,CID8941,...,NaN,44.57,36809.56,11.0,744.970000,Yes,No,Yes,No Information Available,Too many ads
4,fffe43004900440035003000370031003900,Enola Gatto,16,F,100RYB5,Town,No Membership,2015-05-05,Yes,CID5690,...,6.0,349.88,40675.86,8.0,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website


In [36]:
test=test.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [37]:
def fillNan(df, col, value):
    df[col].fillna(value, inplace=True)
    
fillNan(test, 'region_category', test['region_category'].mode()[0])
fillNan(test, 'points_in_wallet', test['points_in_wallet'].mean())
fillNan(test, 'preferred_offer_types', test['preferred_offer_types'].mode()[0])
# fillNan(test, 'joined_through_referral', 'No')
# fillNan(test, 'medium_of_operation', 'Desktop')

In [38]:
for i in test.columns:
  if test[i].dtype=='float64':
    test[i]=test[i].fillna(test[i].mean())
  else:
    test[i]=test[i].fillna(method='ffill')

In [39]:
# test['joining_date']=test['joining_date'].apply(lambda x:int(x.split('-')[0]))
# test.rename(columns={'joining_date':'joining_year'},inplace=True)

test['joining_date'] = pd.to_datetime(test['joining_date'])
test['days_since_joined'] = test['joining_date'].apply(lambda x:(pd.Timestamp('today') - x).days)

test.drop(['joining_date'], inplace=True, axis=1)

In [40]:
test1=pd.get_dummies(test,drop_first=True)

In [41]:
rdpred=xgc.predict(test1)

In [42]:
df2=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/test.csv',usecols=['customer_id'])

In [43]:
df2['churn_risk_score']=rdpred

In [44]:
df2.to_csv('xgb.csv', index=False)
